In [ ]:
# !pip install scikit-learn
import ast
import pandas as pd
from sklearn.model_selection import train_test_split
import sys

# Add the directory to sys.path
sys.path.append('../../biohackathon2024/src/bh24_literature_mining')

# Now import ml_tools
import ml_tools

In [ ]:
import pandas as pd

# sheet_url = 'https://docs.google.com/spreadsheets/d/13BnWa9_r-8aVkbWSnuInvyhI41_WEALMBfT9X3frekI/export?format=csv&gid=79076201'
new_sheet_url =  'https://docs.google.com/spreadsheets/d/13BnWa9_r-8aVkbWSnuInvyhI41_WEALMBfT9X3frekI/export?format=csv&gid=425276317'
new_sheet_url = "/Users/vedran/Downloads/mentions - improved.csv"
# Read the sheet into a DataFrame
df = pd.read_csv(new_sheet_url)

# Optionally, convert 'TRUE' and 'FALSE' to boolean
df['True?'] = df['True?'].apply(lambda x: True if x == 'TRUE' else (False if x == 'FALSE' else x))
df['False?'] = df['False?'].apply(lambda x: True if x == 'TRUE' else (False if x == 'FALSE' else x))


# Display the DataFrame
df


In [ ]:
df.loc[df['False?'] == True, 'NER_Tags'] = None

true_checked_df = df[(df['True?'] == True) | (df['False?'] == True)]

# true_checked_df = df[(df['True?']| (df['False?']]

true_checked_df


In [ ]:
true_checked_df[true_checked_df['False?'] == True]

In [ ]:
true_checked_df = true_checked_df[['Sentence','NER_Tags']]

transformed_df = true_checked_df.rename(columns={
    'Sentence': 'sentence',
    'NER_Tags': 'ner_ines'
})

transformed_df['ner_ines'] = transformed_df['ner_ines'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

transformed_df

In [ ]:
transformed_df['ner_ines'].iloc[0]


In [7]:
# Group by 'sentence' and aggregate 'ner_ines'
grouped_df = transformed_df.groupby('sentence')['ner_ines'].apply(lambda x: [i for i in x if i is not None]).reset_index()


In [ ]:
grouped_df

In [ ]:
grouped_df['ner_ines'] = grouped_df['ner_ines'].apply(
    lambda x: [[item[0], item[1], item[2], 'BT'] for item in x] if x else None
)
grouped_df

In [30]:
grouped_df.iloc[52]

grouped_df.reset_index(drop=True, inplace=True)

In [ ]:
grouped_df.head()

In [ ]:
train_df, test_df = train_test_split(grouped_df, test_size=0.2, random_state=42, shuffle=False)

train_df.head()


In [ ]:
train_df.tail()


In [ ]:
test_df.head()


In [ ]:
# Example usage:
output_folder = '../data/IOB/'

# Convert train, dev, and test dataframes to IOB format
ml_tools.convert_to_IOB_format_from_df(train_df, output_folder, 'train_IOB.tsv')
ml_tools.convert_to_IOB_format_from_df(test_df, output_folder, 'dev_IOB.tsv')

In [ ]:
# Example usage:
train_files = [output_folder+'train_IOB.tsv']
dev_files = [output_folder+'dev_IOB.tsv']

ml_tools.check_integrity_of_files(train_files, dev_files,dev_files)